In [96]:
import pandas as pd
import csv

# check if part of string is subtoken
def is_subtoken(word):
    if word[:2] == "##":
        return True
    else:
        return False


# decode tokens by combining subtokens
def convert_token(token):
    restored_token = []
    token_list = token.split(" ")
    for i in range(len(token_list)):
        if not is_subtoken(token_list[i]) and not token_list[i].isalnum():
            if token_list[i-1].isnumeric():
                restored_token.append(token_list[i])
            else:
                restored_token.append(' ' +token_list[i])

        else:           
            # check if element is start of value or if separated by '.'
            if not is_subtoken(token_list[i]) and token_list[i].isnumeric() and not is_subtoken(token_list[i-1]) and not token_list[i-1].isalnum():
                restored_token.append(token_list[i])          
            # check if element is 
            elif not is_subtoken(token_list[i]) and (i == 0):
                restored_token.append(token_list[i])
            elif not is_subtoken(token_list[i]):
                restored_token.append(" "+ token_list[i])

            # element is a subtoken
            else:
                restored_token.append(token_list[i][2:])

    # return combined value
    return "".join(restored_token)  

In [97]:
# results_path = 'spacy_results.tsv'
# results = pd.read_csv(results_path, sep="\t", dtype=str, keep_default_na=False, na_values=[""], quoting=csv.QUOTE_NONE)
# print(len(results))

encoded_results_path = 'encoded_results.tsv'
encoded_results = pd.read_csv(encoded_results_path, sep="\t", dtype=str, keep_default_na=False, na_values=[""], quoting=csv.QUOTE_NONE)
# print(len(encoded_results))

# for i in range(len(encoded_results)):
#     encoded_results.iat[i,2] = convert_token(encoded_results.iat[i,2])
# Check for missing values in the third column (index 2)
missing_values_mask = encoded_results.iloc[:, 2].isnull()

# Convert tokens only if there are no missing values
for i in range(len(encoded_results)):
    if not missing_values_mask.iat[i]:
        encoded_results.iat[i, 2] = convert_token(encoded_results.iat[i, 2])
    else:
        # Handle missing values (if needed)
        pass

In [98]:
# save results to pandas df
df_submission = pd.DataFrame()
df_submission['Record Number'] = encoded_results.iloc[:,0]
df_submission['Aspect Name'] = encoded_results.iloc[:,1]
df_submission['Aspect Value'] = encoded_results.iloc[:,2]
df_submission['Aspect Value'] = df_submission['Aspect Value'].astype(str)
df_submission['Aspect Name'] = df_submission['Aspect Name'].astype(str)
df_submission = df_submission[df_submission['Aspect Name']!='O']
df_submission = df_submission[df_submission['Aspect Value']!='[CLS]']
df_submission = df_submission[df_submission['Aspect Value']!='[SEP]']
df_submission = df_submission.dropna()
print((len(df_submission)))

159228


In [99]:
df_submission.to_csv("decoded_results.tsv", sep="\t", encoding='utf-8', header=False, index=False)